In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.6.0


In [2]:
img_size = 48
batch_size = 64
datagen_train = ImageDataGenerator(horizontal_flip=True)
train_generator = datagen_train.flow_from_directory("../../content",
                                                    target_size=(img_size,img_size),
                                                    color_mode="grayscale",
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True)
train_generator


Found 0 images belonging to 3 classes.


In [3]:
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils

In [4]:
def readImage(filePath):
    img = Image.open(filePath)
    
    size=(30,40)
    #resize image
    out = img.resize(size)

    # asarray() class is used to convert
    # PIL images into NumPy arrays
    numpydata = asarray(out)
    numpydata = np.repeat(numpydata[:, :, np.newaxis], 3, axis=2)
    # <class 'numpy.ndarray'>
    #print(type(numpydata))

    #  shape
    #print(numpydata.shape)
    return numpydata
print(readImage('data/disease/measles (1).png'))

[[[[ 76  84  95]
   [ 76  84  95]
   [ 76  84  95]]

  [[ 15  15  20]
   [ 15  15  20]
   [ 15  15  20]]

  [[  8   7  11]
   [  8   7  11]
   [  8   7  11]]

  ...

  [[ 16  14  25]
   [ 16  14  25]
   [ 16  14  25]]

  [[ 13  13  22]
   [ 13  13  22]
   [ 13  13  22]]

  [[  8   9  13]
   [  8   9  13]
   [  8   9  13]]]


 [[[ 88  96 107]
   [ 88  96 107]
   [ 88  96 107]]

  [[ 17  17  22]
   [ 17  17  22]
   [ 17  17  22]]

  [[  6   5   9]
   [  6   5   9]
   [  6   5   9]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  24]
   [ 15  13  24]
   [ 15  13  24]]

  [[ 10  10  16]
   [ 10  10  16]
   [ 10  10  16]]]


 [[[ 86  91  98]
   [ 86  91  98]
   [ 86  91  98]]

  [[ 14  13  17]
   [ 14  13  17]
   [ 14  13  17]]

  [[  5   4   7]
   [  5   4   7]
   [  5   4   7]]

  ...

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 15  13  23]
   [ 15  13  23]
   [ 15  13  23]]

  [[ 12  11  18]
   [ 12  11  18]
   [ 12  11  18]]]


 ...


 [[[157 162 

In [5]:
listFiles = os.listdir('data/disease/')
print(listFiles[:5])

['4164443_down-syndrome-down-syndrome-transparent-background-transparent-png.png', 'main-qimg-4c812ebe5c1bdc504625b6961032fd3b.png', 'measles (1).png', 'measles (10).jpg', 'measles (140).jpg']


In [6]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = readImage('data/disease/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[[[243, 243, 243],
         [243, 243, 243],
         [243, 243, 243]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       [[[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        ...,

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]],

        [[242, 242, 242],
         [242, 242, 242],
         [242, 242, 242]]],


       

In [7]:
listFiles = os.listdir('data/health/')
print(listFiles[:5])
for elem in listFiles:
    img = readImage('data/health/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['1 (1).jpeg', '1 (1).jpg', '1 (1).png', '1 (10).jpeg', '1 (10).jpg']


In [8]:
print(len(listCancer))

200


In [10]:
from sklearn.linear_model import LogisticRegression
accs=[]
precisions=[]
recalls=[]
fmeasures=[]

for tst in range(10):
    model = LogisticRegression()
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    ln = int(len(listCancer) * 0.8)
    per = np.random.permutation(len(listCancer))
    for i in range(ln):
        positions += [per[i]]
    for i in range(len(listCancer)):
        if len(listCancer[i].shape) !=4:
            continue
        if listCancer[i].shape[3] == 4:
            continue
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]

    ln = len(x_test) // 2

    x_valid = x_test[ln:]
    y_valid = y_test[ln:]

    x_test = x_test[:ln]
    y_test = y_test[:ln]

    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40*30*9)
    y_train = np.array(y_train)


    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40*30*9)
    y_test = np.array(y_test)

    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40*30*9)
    y_valid = np.array(y_valid)

    model.fit(x_train, y_train)

    pred = model.predict(x_test)
    print(pred)
    print(y_train)
    result=[]
    nr=0
    for i in range(len(pred)):
        val = y_test[i][0]
        if pred[i] == val:
            nr+=1
        result+=[val]
    acc = nr / len(pred)
    precision = precision_score(result, pred)
    recall = recall_score(result, pred)
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred)
    accs+=[acc]
    precisions+=[precision]
    recalls+=[recall]
    fmeasures+=[fmeasure]

print("accs: ", accs)
print("precisions: ", precisions)
print("recalls: ", recalls)
print("fmeasures: ", fmeasures)


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was

[1 1 1 1 1 1 1 1 1 0 0 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1 1 0 1 1 1 1 0 1 0 0 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1 1 0 1 1 0 1 1 1 0 0 0 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1 0 1 1 1 0 1 1 0 1 1 1 0 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[1 1 1 1 1 1 0 0 0 1 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]


c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
accs=  [0.9166666666666666, 0.8333333333333334, 0.7857142857142857, 0.8333333333333334, 0.75, 0.7692307692307693, 0.9230769230769231, 0.8461538461538461, 0.7857142857142857, 0.9090909090909091]
precisions=  [0.8888888888888888, 0.8571428571428571, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8571428571428571]
recalls=  [1.0, 0.8571428571428571, 0.625, 0.75, 0.5714285714285714, 0.75, 0.8888888888888888, 0.7777777777777778, 0.75, 1.0]
fmeasures=  [0.9411764705882353, 0.8571428571428571, 0.7692307692307693, 0.8571428571428571, 0.7272727272727273, 0.8571428571428571, 0.9411764705882353, 0.8750000000000001, 0.8571428571428571, 0.923076923076923]

In [12]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = precisions

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = recalls

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

data = fmeasures

#create 95% confidence interval for population mean weight
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)))

(0.789703506210775, 0.8807593642520953)
(0.9141937541509505, 1.0064411664839703)
(0.6953128784109013, 0.8987347406367177)
(0.810950026000284, 0.9101509318653795)
